# Run a standalone AssistantAgent

AG2 supports running `AssistantAgent` as a standalone agent with the ability to execute simple tasks without the need for interacting with other agents.

In this notebook, we'll enable our assistant agent access to surf the web. We will use the `BrowserUseTool` Tool, for which we need to install the browser-use optional dependency and [playwright](https://playwright.dev/python/docs/intro).

\:\:\:info Requirements

**Warning:**
[`Browser Use`](https://github.com/browser-use/browser-use) requires **Python 3.11 or higher**.

Install `ag2` and `playwright`:
```bash
pip install -U ag2[browser-use]
# Installs Playwright and browsers for all OS
playwright install
# Additional command, mandatory for Linux only
playwright install-deps
```

> **Note:** If you have been using `autogen` or `pyautogen`, all you need to do is upgrade it using:  
> ```bash
> pip install -U autogen[openai,browser-use]
> ```
> or  
> ```bash
> pip install -U pyautogen[browser-use]
> ```
> as `pyautogen`, `autogen`, and `ag2` are aliases for the same PyPI package.  


\:\:\:


In [ ]:
import autogen
from autogen import AssistantAgent
from autogen.tools.experimental.browser_use.browser_use import BrowserUseTool

In [ ]:
import nest_asyncio

nest_asyncio.apply()

## Set your API Endpoint

The [`LLMConfig.from_json`](https://docs.ag2.ai/docs/api-reference/autogen/config_list_from_json#config-list-from-json) method loads a list of configurations from an environment variable or a JSON file.

In [ ]:
llm_config = autogen.LLMConfig.from_json(path="OAI_CONFIG_LIST", temperature=0.8, timeout=600).where(
    tags=["gpt-4o-mini"]
)

# Configure your assistant agent

Here we will configure two assistant agents:
1. x_assistant, tasked with exploring the trending topics on X (formally Twitter)
2. arxiv_researcher, tasked with discovery of papers that align with the hot topic of the day

We will set the browser configuration to not run headless, this will open the browser as a window so you can see it in action.

In [ ]:
x_assistant = AssistantAgent(name="x_assistant", llm_config=llm_config)

arxiv_researcher = AssistantAgent(name="arxiv", llm_config=llm_config)

browser_use_tool = BrowserUseTool(
    llm_config=llm_config,
    browser_config={"headless": False},
)

## Running the assistant agents

Let's run our x_assistant to discover the hot topic of the day.

To be able to do this we give our assistant the capability to browse the web using a `BrowserUseTool` Tool.

In [ ]:
hot_topic_res = x_assistant.run(
    "Find out today's hot topic and an influencer who is talking about it on X using a web search",
    tools=browser_use_tool,
    user_input=False,
)

print(hot_topic_res.summary)

After discovering the hot topic, let's run the discovery of papers that align with the topic

In [ ]:
paper_abstract = arxiv_researcher.run(
    "Get the abstract of a relevant paper based on:\n" + hot_topic_res.summary,
    user_input=False,
)

print(paper_abstract.summary)

Now, let's create an X post using our x_assistant

In [ ]:
# Scenario 1. This task requires x_assistant's past state
post = x_assistant.run(
    "Create an X post based on the hot topic and the following and mention the influencer:\n" + paper_abstract.summary,
    user_input=False,
)

print(post.summary)

Finally, let's ask our x_assistant who should we follow on X

In [ ]:
# Scenario 2. Doing another task that does not require history or past state

influencer_choice = x_assistant.run(
    "Find a influencer I should follow on Twitter by searching the web",
    clear_history=True,
    tools=browser_use_tool,
    user_input=False,
)

print(influencer_choice.summary)